In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: [Mar 24, 11:59 pm](https://github.com/UBC-CS/cpsc330-2024W2?tab=readme-ov-file#deliverable-due-dates-tentative).**

## Imports

In [2]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

<!-- BEGIN QUESTION -->

<div class="alert alert-info">
    
## Submission instructions
<hr>
rubric={points}

**Please be aware that this homework assignment requires installation of several packages in your course environment. It's possible that you'll encounter installation challenges, which might be frustrating. However, remember that solving these issues is not wasting time but it is an essential skill for anyone aspiring to work in data science or machine learning.**

Follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2024W2/blob/main/docs/homework_instructions.md). 

**You may work in a group on this homework and submit your assignment as a group.** Below are some instructions on working as a group.  
- The maximum group size is 2. 
- Use group work as an opportunity to collaborate and learn new things from each other. 
- Be respectful to each other and make sure you understand all the concepts in the assignment well. 
- It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. 
- You can find the instructions on how to do group submission on Gradescope [here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members).


When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission.
4. Make sure that the plots and output are rendered properly in your submitted file. 
5. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb. If the pdf or html also fail to render on Gradescope, please create two files for your homework: hw6a.ipynb with Exercise 1 and hw6b.ipynb with Exercises 2 and 3 and submit these two files in your submission.  
</div>

_Points:_ 2

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [3]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [4]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [5]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words using `glove_wiki_vectors` embeddings.

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 2

In [6]:
l = ["burger", "boxing", "statistics", "iphone"]

In [7]:
glove_wiki_vectors.most_similar(l[0])

[('burgers', 0.70148104429245),
 ('pizza', 0.6977595686912537),
 ('kfc', 0.6645278334617615),
 ('taco', 0.6631638407707214),
 ('mcdonald', 0.6614505648612976),
 ('hamburger', 0.6495352387428284),
 ('fries', 0.6424607634544373),
 ('sandwich', 0.635134756565094),
 ('hamburgers', 0.6286556720733643),
 ('whopper', 0.6201803088188171)]

In [8]:
glove_wiki_vectors.most_similar(l[1])

[('wrestling', 0.7694606184959412),
 ('heavyweight', 0.7335327863693237),
 ('welterweight', 0.6844874620437622),
 ('middleweight', 0.6734765768051147),
 ('boxers', 0.6718363761901855),
 ('kickboxing', 0.668809175491333),
 ('judo', 0.6509308815002441),
 ('ibf', 0.6399338245391846),
 ('featherweight', 0.6371622085571289),
 ('wbc', 0.6348312497138977)]

In [9]:
glove_wiki_vectors.most_similar(l[2])

[('figures', 0.7243505716323853),
 ('report', 0.6801027655601501),
 ('data', 0.6705882549285889),
 ('statistic', 0.6542533040046692),
 ('bureau', 0.6409845352172852),
 ('according', 0.64072185754776),
 ('estimates', 0.6405200958251953),
 ('statistical', 0.6386309266090393),
 ('survey', 0.6346055269241333),
 ('gdp', 0.6314313411712646)]

In [10]:
glove_wiki_vectors.most_similar(l[3])

[('ipad', 0.9220294952392578),
 ('ipod', 0.8369015455245972),
 ('smartphone', 0.7984607219696045),
 ('android', 0.7606974840164185),
 ('app', 0.749142587184906),
 ('apps', 0.7407808303833008),
 ('ios', 0.7348269820213318),
 ('smartphones', 0.7175774574279785),
 ('handsets', 0.7168352007865906),
 ('blackberry', 0.7162630558013916)]

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [11]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 2

In [12]:
for w1, w2 in word_pairs:
    print(glove_wiki_vectors.similarity(w1,w2))

0.7000272
0.546276
0.6432488
0.7552732
0.8798075
0.076719455


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Representation of all words in English
rubric={points}

**Your tasks:**

1. The vocabulary size of Wikipedia embeddings is quite large. The `test_words` list below contains a few new words (called neologisms) and biomedical domain-specific abbreviations. Write code to check whether `glove_wiki_vectors` have representation for these words or not. 
> If a given word `word` is in the vocabulary, `word in glove_wiki_vectors` will return True. 

In [13]:
test_words = [
    "covididiot",
    "fomo",
    "frenemies",
    "anthropause",
    "photobomb",
    "selfie",
    "pxg",  # Abbreviation for pseudoexfoliative glaucoma
    "pacg",  # Abbreviation for primary angle closure glaucoma
    "cct",  # Abbreviation for central corneal thickness
    "escc",  # Abbreviation for esophageal squamous cell carcinoma
]

<div class="alert alert-warning">

Solution_1_3
    
</div>

_Points:_ 2

In [14]:
for i in test_words:
    print(i in glove_wiki_vectors)

False
False
True
False
False
False
False
False
True
True


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Stereotypes and biases in embeddings
rubric={points}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore this. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

In [15]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

Examples of using analogy to explore biases and stereotypes.  

In [16]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


In [17]:
glove_wiki_vectors.similarity("aboriginal", "success")

0.14283238

In [18]:
glove_wiki_vectors.similarity("white", "success")

0.35182396

<div class="alert alert-warning">

Solution_1_4
    
</div>

_Points:_ 4

In [19]:
analogy("american", "wealthy", "mexican")

american : wealthy :: mexican : ?


,Analogy word,Score
0,affluent,0.540980
1,wealthiest,0.530729
2,businessmen,0.514996
3,salinas,0.514892
4,well-to-do,0.508769
5,gortari,0.499227
6,mexico,0.492073
7,landlords,0.487137
8,traffickers,0.487094
9,fabulously,0.484275


In [20]:
analogy("he","king","she")

he : king :: she : ?


,Analogy word,Score
0,queen,0.835802
1,princess,0.748912
2,prince,0.707642
3,daughter,0.701513
4,elizabeth,0.699587
5,mother,0.681290
6,sister,0.672442
7,throne,0.658172
8,monarch,0.651568
9,father,0.650617


In [21]:
analogy("thin","beautiful","fat")

thin : beautiful :: fat : ?


,Analogy word,Score
0,wonderful,0.619958
1,fabulous,0.579185
2,lovely,0.578286
3,gorgeous,0.549044
4,love,0.539363
5,crazy,0.536324
6,fun,0.529835
7,cute,0.518925
8,beloved,0.518165
9,kids,0.516160


In [22]:
analogy("urban","educated","rural")

urban : educated :: rural : ?


,Analogy word,Score
0,illiterate,0.531743
1,schooled,0.514549
2,wealthy,0.509559
3,uneducated,0.502137
4,affluent,0.497697
5,enrolled,0.490494
6,parents,0.482439
7,disadvantaged,0.477240
8,emigrated,0.475279
9,lived,0.472891


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Discussion
rubric={points}

**Your tasks:**
1. Discuss your observations from 1.4. Are there any worrisome biases in these embeddings trained on Wikipedia?   
2. Give an example of how using embeddings with biases could cause harm in the real world.

<div class="alert alert-warning">

Solution_1_5
    
</div>

_Points:_ 4

1. We can seem some biases in the first and last analogies. In the first analogy of american:wealthy, while we see that most of the terms for mexican are positive, the term traffickers does show up in the top 10. In addition, for the analogy of urban:educated, we see that the most analogous term for rural is illiterate.
2. Biases such as the ones above can have long lasting impact. The association of trafficking with mexicans can lead to the formation of negative stereotypes that affect a large group of people. In addition, the view that rural people are illiterate might lead to reduced job opportunities affecting the economic well-being of rural communities.

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The goal of topic modeling is discovering high-level themes in a large collection of texts. 

In this homework, you will explore topics in [the 20 newsgroups text dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [23]:
from sklearn.datasets import fetch_20newsgroups

In [24]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [25]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points}

Preprocessing is a crucial step before carrying out topic modeling and it markedly affects topic modeling results. In this exercise, you'll prepare the data using [spaCy](https://spacy.io/) for topic modeling. 

**Your tasks:** 

- Write code using [spaCy](https://spacy.io/) to preprocess the `text` column in the given dataframe `df` and save the processed text in a new column called `text_pp` within the same dataframe.

If you do not have spaCy in your course environment, you'll have to [install it](https://spacy.io/usage) and download the pretrained model en_core_web_md. 

`python -m spacy download en_core_web_md`


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are likely to be more informative for the given task. Some common text preprocessing steps for topic modeling include: 
- getting rid of slashes, new-line characters, or any other non-informative characters
- sentence segmentation and tokenization      
- replacing urls, email addresses, or numbers with generic tokens such as "URL",  "EMAIL", "NUM". 
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization


> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://universaldependencies.org/u/pos/) is the list of part-of-speech tags used in spaCy. 

> You may have to use regex to clean text before passing it to spaCy. Also, you might have to go back and forth between preprocessing in this exercise and and topic modeling in Exercise 2 before finalizing preprocessing steps. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you finalize the preprocessing part, you might want to save the preprocessed data in a CSV and work with this CSV so that you don't run the preprocessing part every time you run the notebook. 
 


In [26]:
import spacy
nlp = spacy.load("en_core_web_md", disable=["parser", "ner"])

<div class="alert alert-warning">

Solution_2_1
    
</div>

_Points:_ 8

In [27]:
# Credit: https://github.com/UBC-CS/cpsc330-2024W2/blob/main/lectures/202-203-Giulia-lectures/18_natural-language-processing.ipynb

def preprocess(
    doc,
    min_token_len=2,
    irrelevant_pos=["ADV", "PRON", "CCONJ", "PUNCT", "PART", "DET", "ADP", "SPACE"],
):
    """
    Given text, min_token_len, and irrelevant_pos carry out preprocessing of the text
    and return a preprocessed string.

    Parameters
    -------------
    doc : (spaCy doc object)
        the spacy doc object of the text
    min_token_len : (int)
        min_token_length required
    irrelevant_pos : (list)
        a list of irrelevant pos tags

    Returns
    -------------
    (str) the preprocessed text
    """

    clean_text = []

    for token in doc:
        if (
            token.is_stop == False  # Check if it's not a stopword
            and len(token) > min_token_len  # Check if the word meets minimum threshold
            and token.pos_ not in irrelevant_pos
        ):  # Check if the POS is in the acceptable POS tags
            lemma = token.lemma_  # Take the lemma of the word
            clean_text.append(lemma.lower())
    return " ".join(clean_text)

In [28]:
df["text_pp"] = [preprocess(text) for text in nlp.pipe(df["text"])]

In [29]:
df.iloc[2:6]

,text,target,target_name,text_pp
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support idea have newsgroup asp...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,follow problem bug appreciate help create wind...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late upi foreign ministry spokesman ferhat ata...
5,"Hi,\n I'd like to subscribe to Leadership Ma...",5,soc.religion.christian,like subscribe leadership magazine wonder disk...


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification
rubric={points}

**Your tasks:**

- Outline the preprocessing steps you carried out in the previous exercise (bullet point format is fine), providing a brief justification when necessary. 

> You might want to wait to answer this question till you are done with Exercise 2 and you have finalized the preprocessing steps in 2.1. 

<div class="alert alert-warning">

Solution_2_2
    
</div>

_Points:_ 2

For Each Document Text:
- Iterate through the tokens
- Remove tokens present in the list of stop words as these words do not add meaning
- Remove tokens less than 2 characters long since these tokens are unlikely to add much semantic meaning
- Remove tokens that are adverbs since they just modify existing verbs and can dilute their meaning, pronouns since they lack meaning as they refer to another noun as well as conjunctions, punctuations, particles, determiners, adpositions, and spaces that serve purposes in grammar but do not add meaning in the context of natural language processing.
- Add the lemma of the token to identify different words with the same stem that share the same meaning.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points}

**Your tasks:**

1. Build LDA models on the preprocessed data using using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html) and random state 42. Experiment with a few values for the number of topics (`n_components`). Pick a reasonable number for the number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

_Points:_ 4

Since we are fetching text data from 8 news categories, it seems reasonable for our LDA model to also have `n_components`=8. This number also creates topics that are general enough to be meaningful, but also specific enough that the topics make sense.

In [30]:
vec = CountVectorizer(stop_words="english")
X = vec.fit_transform(df['text_pp'])

In [31]:
from sklearn.decomposition import LatentDirichletAllocation
import mglearn

n_topics = 8
lda = LatentDirichletAllocation(
    n_components=n_topics, learning_method="batch", max_iter=10, random_state=42
)
document_topics = lda.fit_transform(X)

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association
rubric={points}

**Your tasks:**
1. For the number of topics you picked in the previous exercise, show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

_Points:_ 5

In [32]:
import mglearn

sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vec.get_feature_names_out())

mglearn.tools.print_topics(
    topics=range(n_topics),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=5,
    n_words=15,
)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
file          game          people        window        team          
image         play          armenian      widget        player        
program       win           israel        server        game          
entry         period        turkish       motif         hockey        
use           goal          jews          edu           league        
edu           team          say           use           year          
format        season        armenians     application   play          
graphic       power         government    run           season        
thank         point         kill          available     nhl           
know          second        state         set           new           
color         shot          israeli       include       fan           
send          16            war           file          good          
line  

The Topic Assignments are:

| **Topic Number** | **Topic Label**   |
|-------------------|-------------------|
| Topic 0           | Files and Graphics |
| Topic 1           | Sports and Games  |
| Topic 2           | Global Politics |
| Topic 3           | Software and Applications |
| Topic 4           | Hockey Teams and Leagues |
| Topic 5           | Thoughts and Opinions |
| Topic 6           | Law and Rights   |
| Topic 7           | Religion |


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association
rubric={points}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

_Points:_ 5

As we can see, most topic assignments are correct even though the target column shows different numbers. This is because the same topic is assigned a different number in the LDA model. The only document that appears to be assigned the wrong topic is the third document which is wrongly assigned the topic Law and Rights. This might be due to similarity in the vocabulary used in this row and other documents in the wrong topic or a lack of similar words in other documents in the correct topic assignment.

In [33]:
for i in range(5):
     print("Document %s: Topic %s" % (i,np.argmax(document_topics[i])))
df.head(5)

Document 0: Topic 6
Document 1: Topic 0
Document 2: Topic 0
Document 3: Topic 6
Document 4: Topic 2


,text,target,target_name,text_pp
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns,know read u.s.c. 922 wonder help u.s.c. 922 pr...
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics,bad question ref list algorithm think bit hard...
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support idea have newsgroup asp...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,follow problem bug appreciate help create wind...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late upi foreign ministry spokesman ferhat ata...


<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

rubric={points}

1. Briefly explain how content-based filtering works in the context of recommender systems. 
2. Discuss at least two negative consequences of recommender systems.
3. What is transfer learning in natural language processing? Briefly explain.     

<div class="alert alert-warning">

Solution_3
    
</div>

_Points:_ 6

1. In content-based filtering, given user ratings of objects and various object features, we construct separate regression models for each user using the object features as the input variables and the user rating as the response. The model for a particular user is then used to predict the ratings that the user would give to other objects and recommend objects with the highest predicted ratings.
2. Two negative consequences of recommender systems include the forming of content or information bubbles where users only see content similar to their previous history and find it hard to explore new topics. In addition, recommender systems might rely on historical data which often contains biases and as a result might propagate them.
3. Transfer learning is the use of a model trained on one task for another. After the initial training, the model can then be fine tuned to accomplish the second task. This has many advantages, including having a model pre-trained on large amounts of data, saving both time and resources.

<!-- END QUESTION -->

<br><br><br><br>

**Before submitting your assignment, please make sure you have followed all the instructions in the Submission instructions section at the top.** 

![](img/eva-well-done.png)